# Setup

In [1]:
import os
import openai
import sys
import pandas as pd
# from datasets import Dataset
sys.path.append('../..')

# Load environment
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# Set the OpenAI API key
openai.api_key  = os.environ['OPENAI_API_KEY']

In [2]:
# define llm
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o", temperature=0.5)

/home/kaniagalih/pricai/DREAM_CS-TUTORIAL-PRICAI-2024/pricai_venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Web Scrape

In [3]:
import asyncio
from urllib.parse import urljoin
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

async def fetch_links_from_page(url):
    async with async_playwright() as p:
        # Launch a browser instance
        browser = await p.chromium.launch()
        page = await browser.new_page()
        # Go to the specified URL
        await page.goto(url)
        # Get the content of the page
        html = await page.content()
        # Close the browser
        await browser.close()

        # Use BeautifulSoup to parse the HTML content
        soup = BeautifulSoup(html, "html.parser")
        links = []
        # Extract all <a> tags with href attributes
        for link in soup.find_all("a", href=True):
            href = link.get("href")
            # Construct full URL using urljoin
            full_url = urljoin(url, href)
            links.append(full_url)

        return list(set(links)) # Remove duplicates

# Example usage
url_to_scrape = "https://pricai.org/2024/"  # Replace with the desired URL

# Use await directly in a Jupyter notebook
links = await fetch_links_from_page(url_to_scrape)

# Print extracted full links
print("Extracted Full Links:")
for link in links:
    print(link)


Extracted Full Links:
https://pricai.org/2024/index.php/calls/call-for-domestic-volunteers-jp
https://pricai.org/2024/index.php/calls/call-for-posters
https://pricai.org/2024/index.php/programs/programs-workshops
https://pricai.org/2024/index.php/organization/organising-committee
https://pricai.org/2024/index.php/programs/technical-sessions
https://pricai.org/2024/index.php/registration
https://pricai.org/2024/#top
https://pricai.org/2024/index.php/programs/panel-sessions
https://pricai.org/2024/index.php/attending
https://pricai.org/2024/index.php/programs/keynote-speakers
https://pricai.org/2024/index.php/calls/calls-for-workshops
https://pricai.org/2024/index.php/submissions/submission
https://pricai.org/2024/index.php/attending/local-attractions
https://pricai.org/2024/index.php/calls/call-for-tutorials2
https://pricai.org/2024/index.php/programs/program-at-a-glance
https://pricai.org/2024/index.php/calls/call-for-sponsorship
https://pricai.org/2024/index.php/programs/tutorials
htt

In [4]:
links

['https://pricai.org/2024/index.php/calls/call-for-domestic-volunteers-jp',
 'https://pricai.org/2024/index.php/calls/call-for-posters',
 'https://pricai.org/2024/index.php/programs/programs-workshops',
 'https://pricai.org/2024/index.php/organization/organising-committee',
 'https://pricai.org/2024/index.php/programs/technical-sessions',
 'https://pricai.org/2024/index.php/registration',
 'https://pricai.org/2024/#top',
 'https://pricai.org/2024/index.php/programs/panel-sessions',
 'https://pricai.org/2024/index.php/attending',
 'https://pricai.org/2024/index.php/programs/keynote-speakers',
 'https://pricai.org/2024/index.php/calls/calls-for-workshops',
 'https://pricai.org/2024/index.php/submissions/submission',
 'https://pricai.org/2024/index.php/attending/local-attractions',
 'https://pricai.org/2024/index.php/calls/call-for-tutorials2',
 'https://pricai.org/2024/index.php/programs/program-at-a-glance',
 'https://pricai.org/2024/index.php/calls/call-for-sponsorship',
 'https://pric

In [5]:
import asyncio
from urllib.parse import urljoin
from playwright.async_api import async_playwright
from bs4 import BeautifulSoup

async def scrape_information_from_page(url):
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto(url)
        html = await page.content()
        await browser.close()

        soup = BeautifulSoup(html, "html.parser")
        title = soup.title.string if soup.title else 'No Title'
        paragraphs = [p.get_text() for p in soup.find_all('p')]

        return {
            'url': url,
            'title': title,
            'paragraphs': paragraphs
        }

async def main(url):
    links = await fetch_links_from_page(url)
    all_scraped_data = []
    for link in links:
        try:
            scraped_data = await scrape_information_from_page(link)
            all_scraped_data.append(scraped_data)
            print(f"Scraped data from: {link}")
        except Exception as e:
            print(f"Failed to scrape {link}: {e}")

    return all_scraped_data

# Wrapper for Jupyter compatibility
url_to_scrape = "https://pricai.org/2024/"

# Run the asynchronous main function in Jupyter
scraped_data = await main(url_to_scrape)

# Print the scraped data
for data in scraped_data:
    print(f"URL: {data['url']}")
    print(f"Title: {data['title']}")
    print(f"Paragraphs: {data['paragraphs'][:2]}")
    print("---")


Scraped data from: https://pricai.org/2024/index.php/calls/call-for-domestic-volunteers-jp
Scraped data from: https://pricai.org/2024/index.php/calls/call-for-posters
Scraped data from: https://pricai.org/2024/index.php/programs/programs-workshops
Scraped data from: https://pricai.org/2024/index.php/organization/organising-committee
Scraped data from: https://pricai.org/2024/index.php/programs/technical-sessions
Scraped data from: https://pricai.org/2024/index.php/registration
Scraped data from: https://pricai.org/2024/#top
Scraped data from: https://pricai.org/2024/index.php/programs/panel-sessions
Scraped data from: https://pricai.org/2024/index.php/attending
Scraped data from: https://pricai.org/2024/index.php/programs/keynote-speakers
Scraped data from: https://pricai.org/2024/index.php/calls/calls-for-workshops
Scraped data from: https://pricai.org/2024/index.php/submissions/submission
Scraped data from: https://pricai.org/2024/index.php/attending/local-attractions
Scraped data fr

In [6]:
scraped_data

[{'url': 'https://pricai.org/2024/index.php/calls/call-for-domestic-volunteers-jp',
  'title': 'Call for Domestic Volunteers (JP)',
  'paragraphs': ['\xa0',
   '開催概要',
   '- イベント名: PRICAI 2024（第20回 太平洋国際人工知能会議）/ PRIMA 2024（第27回 マルチエージェントシステムの原則と実践に関する国際会議）',
   '- 開催日: 2024年11月18日～2024年11月24日',
   '- 開催場所: 京都大学百周年時計台記念館',
   '- 主催: PRICAI組織委員会 / PRIMA組織委員会',
   '- 世界中からAI・マルチエージェントシステムの研究者、技術者、企業関係者が集まる国際学術会議です。',
   '\xa0',
   'ボランティア募集概要',
   '募集期間',
   '- 2024年11月18日 ～ 2024年11月24日',
   '\xa0',
   '募集人数',
   '- 50名程度',
   '\xa0',
   '活動内容',
   '会議期間中、以下のようなサポート業務に従事していただきます。',
   '- 受付対応（参加者への案内、バッジの配布、資料提供など）',
   '- 会場設営・撤去のサポート',
   '- 参加者の誘導およびセッション会場の管理',
   '- スピーカーやゲストのサポート',
   '- オンラインセッションの技術サポート',
   '- その他、会議運営に関する補助業務',
   '\xa0',
   '応募資格',
   '- 18歳以上の方（学生、社会人問いませんが、学生の方が望ましい）',
   '- 英語でのコミュニケーションが可能な方（国際会議のため、英語を使った案内やサポート業務が含まれます）',
   '- 人工知能やマルチエージェントシステムに興味がある方歓迎',
   '- 学会や国際会議の運営に関心がある方',
   '- チームでの作業に協力的な方',
   '\xa0',
   'ボランティア期間',
   '- 2024年11月17日 ～ 2024年

# RAG Pipeline

In [7]:
from langchain.schema import Document

# Initialize a dictionary to store links and content
link_content_map = {}

# Populate the link_content_map while creating the documents
documents = [
    Document(
        page_content=" ".join(data['paragraphs']),
        metadata={"url": data['url'], "title": data['title']}
    )
    for data in scraped_data
]

In [8]:
# text splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=300,
    length_function=len,
    is_separator_regex=False,
)

chunk_docs = text_splitter.split_documents(documents)

In [10]:
# Text embedding
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [11]:
# Vector Store
from langchain_chroma import Chroma

db_chroma = Chroma.from_documents(chunk_docs, embeddings)

In [16]:
# Retriever
retriever = db_chroma.as_retriever(search_type="mmr")

In [12]:
# Function to print contents of chunk_docs
def print_stored_documents(chunk_docs, limit=20):
    print("Contents of the Chroma Vector Database:")
    print("{:<100} {:<200}".format("Link", "Generated from Web Scraping"))  # Header
    print("=" * 300)  # Separator

    # Iterate through the chunked documents and print their content
    for idx, doc in enumerate(chunk_docs):
        if idx >= limit:  # Limit to the first 'limit' entries
            break

        # Extract URL and content from metadata and page content
        url = doc.metadata.get("url", "No URL")
        content = doc.page_content  # The content generated from web scraping

        # Print the URL and the content
        print("{:<100} {:<200}".format(url, content[:197] + '...' if len(content) > 200 else content))

# Call the function to print the stored documents
print_stored_documents(chunk_docs)

Contents of the Chroma Vector Database:
Link                                                                                                 Generated from Web Scraping                                                                                                                                                                             
https://pricai.org/2024/index.php/calls/call-for-domestic-volunteers-jp                              開催概要 - イベント名: PRICAI 2024（第20回 太平洋国際人工知能会議）/ PRIMA 2024（第27回 マルチエージェントシステムの原則と実践に関する国際会議） - 開催日: 2024年11月18日～2024年11月24日 - 開催場所: 京都大学百周年時計台記念館 - 主催: PRICAI組織委員会 / PRIMA組織委員会 - 世界中からAI・マルチエージェントシステム...
https://pricai.org/2024/index.php/calls/call-for-posters                                             The CFP flyer has been published! We are pleased to announce that the 21st Pacific Rim International Conference on Artificial Intelligence (PRICAI 2024) is opening a call for posters on the main to...
https://pricai.org/2024/index.php/programs/programs-wo

In [13]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# define prompt template
answer_prompt = """
You are an intelligent system designed to assist users with their questions about PRICAI 2024 and provide helpful, accurate, and natural-sounding responses. Follow ONLY the provided context to answer the question, thinking step-by-step before responding.
As "AI Assistant," your task is to provide clear and concise answers using the available context. If there is no relevant information, respond with "Hmm, I don't know the answer to that, its out of my resource"
Refuse to answer any unrelated questions, never break character, and always base your response on the given context.
PAY ATTENTION if the user asks about products, or brands other than emos. Answer directly with "Hmm, I don't know the answer to that, its out of my resource"

Do not create an answer if:

- The answer cannot be determined from the context alone. In this case, respond with "Hmm, I don't know the answer to that, its out of my resource"
- The context is empty. Respond with "Hmm, I don't know the answer to that, its out of my resource"

Steps to answer the question:
1. Check the user's question for any slang, abbreviations, or synonyms. Interpret these if needed.
2. Understand the use of word variations, such as timeline, schedule is the same as improtant date.
3. Verify if the question relates to the PRICAI 2024 data source. If not, inform the user that the question is unrelated to the data source.
4. If the question relates to PRICAI 2024, generate an answer based on the data source.
5. Review the generated response to ensure accuracy.

The following is a conversation between a user and an AI:

Question: {messages}
Answer: {context}

Provide a clear answer with steps or bullet points if needed, without any line breaks. Ensure the response is neat, organized, and easy to understand, giving detailed and helpful responses.
"""

# Chat prompt template
question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", answer_prompt),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [14]:
from langchain_core.messages import HumanMessage
from langchain_community.chat_message_histories import ChatMessageHistory
message_history = ChatMessageHistory()
# Function to query the vector store and generate a response
def generate_response(query):
    # Retrieve similar documents
    docs = retriever.get_relevant_documents(query, k=10)

    message_history.add_message(HumanMessage(content=query))

    # Use the documents as context for the LLM
    response = document_chain.invoke(
        {
            "context": docs,
            "messages": [
                HumanMessage(content=query)
            ],
        }
    )

    return response

# Test

In [ ]:
# query
query = "where is Pricai converence held?"
response = generate_response(query)

print(response)

/home/kaniagalih/pricai/DREAM_CS-TUTORIAL-PRICAI-2024/pricai_venv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


PRICAI 2024 will be held in person in Kyoto, Japan.


In [ ]:
# query
query = "what can I prepare to attent the Pricai conference?"
response = generate_response(query)

print(response)

The CFS flyer has been published! PRICAI 2024 is an international conference aimed at sharing the latest research and advances in artificial intelligence in the Pacific Rim and facilitating dialogue between academia and industry. To attend the conference, you can prepare by doing the following:

- Reserve your accommodation as soon as possible due to the high demand during the conference period in Japan. There are several options near the conference venue, such as those near Kyoto station.
- Consider the sponsorship opportunities if you represent an organization, as they offer various packages like Platinum and Gold sponsors with different benefits.
- Be aware of the application deadline on October 31, 2024, and the payment deadline on November 11, 2024. All deadlines are at the end of the day specified, anywhere on Earth (UTC-12).
- Plan to engage in knowledge exchange, networking, and developing new ideas with researchers, engineers, students, and industry leaders.

These preparation

In [ ]:
# query
query = "when last call for the proposal and paper submited?"
response = generate_response(query)

print(response)

The deadline for proposal and paper submissions for PRICAI 2024 was originally set for July 10, 2024, but it was extended to July 25, 2024, at 11:59 UTC-12.


In [ ]:
# query
query = "What is pricai, and why they make an event every year, what they expected from that event?"
response = generate_response(query)

print(response)

PRICAI 2024 is an international conference aimed at sharing the latest research and advances in artificial intelligence in the Pacific Rim and facilitating dialogue between academia and industry. The conference brings together researchers, engineers, students, and industry leaders for knowledge exchange, networking, and the development of new ideas. It is expected to host a lively discussion on AI from young researchers, including students, to prominent researchers from different countries. The event provides an opportunity to appeal to a wider audience and enhance a company's brand value while demonstrating commitment to the cutting-edge AI community.


In [21]:
# query
query = "I want to cook pisang goreng, please make the step and recipe"
response = generate_response(query)

print(response)

Hmm, I don't know the answer to that, its out of my resource


In [22]:
# query
query = "Who is Prof. Bo An, Nanyang Technological University, Singapore also what his contribution in Pricai 2024?"
response = generate_response(query)

print(response)

- Prof. Bo An is a President’s Council Chair Professor in Computer Science and Co-Director of the Artificial Intelligence Research Institute at Nanyang Technological University, Singapore. - He is a keynote speaker at PRICAI 2024, presenting on "From Algorithmic and RL-based to LLM-powered Agents." - His research focuses on AI, multiagent systems, computational game theory, reinforcement learning, and optimization, with applications in infrastructure security, sustainability, and e-commerce.


In [23]:
# query
query = "give me the schedule for pricai 2024"
response = generate_response(query)

print(response)

The CFS flyer has been published! PRICAI 2024 is an international conference aimed at sharing the latest research and advances in artificial intelligence in the Pacific Rim. Here are some important dates: Application Deadline: October 31, 2024; Payment Deadline: November 11, 2024. All deadlines are at the end of the day specified, anywhere on Earth (UTC-12). PRICAI 2024 and PRIMA 2024 will be held in collaboration on the same day at the same conference venue.


# RAGAS Evaluation

In [284]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from ragas.embeddings import LangchainEmbeddingsWrapper

generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [285]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(documents, testset_size=10)

Applying [SummaryExtractor, HeadlinesExtractor]:   0%|          | 0/48 [00:00<?, ?it/s]

Applying EmbeddingExtractor:   0%|          | 0/24 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/24 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, KeyphrasesExtractor, TitleExtractor]:   0%|          | 0/204 [00:00<?, ?it/s]

Applying CosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Applying SummaryCosineSimilarityBuilder:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating common themes:   0%|          | 0/1 [00:00<?, ?it/s]

Generating common_concepts:   0%|          | 0/1 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

In [291]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What abstract concepts can be derived from a c...,"[, , , , , ]","From a context with no explicit information, a...",AbstractQuerySynthesizer
1,What abstract concepts can be derived from a c...,"[, , , , , ]","From a context with no explicit information, a...",AbstractQuerySynthesizer
2,What challenges occur when generating abstract...,"[, , , , , ]",Generating abstract questions from no context ...,AbstractQuerySynthesizer
3,How do the themes and applications of artifici...,[The CFP flyer for PRICAI 2024 has been publis...,The themes and applications of artificial inte...,ComparativeAbstractQuerySynthesizer
4,How do the themes and focus areas of the Pacif...,[The CFP flyer for PRICAI 2024 has been publis...,The themes and focus areas of the Pacific Rim ...,ComparativeAbstractQuerySynthesizer
5,How do the themes and focus areas of the Pacif...,[The CFP flyer for PRICAI 2024 has been publis...,The Pacific Rim International Conference on Ar...,ComparativeAbstractQuerySynthesizer
6,Where to find info on free registations?,[News 2024/09/29 Information on free registrat...,Information on free registrations for workshop...,SpecificQuerySynthesizer
7,What real-world AI applications in education d...,[Workshop #3 : The 1st International Workshop ...,The real-world AI applications in education di...,SpecificQuerySynthesizer
8,What are the advantages of using diffusion mod...,[Tutorial # 3 : Overview of Domain-Specific RA...,Diffusion models offer advantages over traditi...,SpecificQuerySynthesizer
9,What are some of the key topics related to mul...,[The 25th International Conference on Principl...,The text does not provide specific key topics ...,SpecificQuerySynthesizer


In [287]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o"))
evaluator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

In [290]:
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, SemanticSimilarity
from ragas import evaluate

metrics = [
    LLMContextRecall(llm=evaluator_llm),
    FactualCorrectness(llm=evaluator_llm),
    Faithfulness(llm=evaluator_llm),
    SemanticSimilarity(embeddings=evaluator_embeddings)
]
results = evaluate(dataset=dataset, metrics=metrics)

AttributeError: 'list' object has no attribute 'get_sample_type'